# ABL1 complex modeling

This notebook explores the template selection for modeling tyrosine kinase inhibitors into ABL1 using functionalities from KinoML.

In [1]:
from kinoml.modeling.OpenEyeModeling import select_structure
import klifs_utils

In [2]:
# smiles from chembl
uniprot_id = 'P00519'
tkis = {'axitinib': {'smiles': 'CNC(=O)c1ccccc1Sc1ccc2c(/C=C/c3ccccn3)n[nH]c2c1'},
        'bosutinib': {'smiles': 'COc1cc(Nc2c(C#N)cnc3cc(OCCCN4CCN(C)CC4)c(OC)cc23)c(Cl)cc1Cl'},
        'dasatinib': {'smiles': 'Cc1nc(Nc2ncc(C(=O)Nc3c(C)cccc3Cl)s2)cc(N2CCN(CCO)CC2)n1'},
        'imatinib': {'smiles': 'Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc(-c2cccnc2)n1'},
        'nilotinib': {'smiles': 'Cc1cn(-c2cc(NC(=O)c3ccc(C)c(Nc4nccc(-c5cccnc5)n4)c3)cc(C(F)(F)F)c2)cn1'},
        'ponatinib': {'smiles': 'Cc1ccc(C(=O)Nc2ccc(CN3CCN(C)CC3)c(C(F)(F)F)c2)cc1C#Cc1cnc2cccnn12'},
        'erlotinib': {'smiles': 'C#Cc1cccc(Nc2ncnc3cc(OCCOC)c(OCCOC)cc23)c1'},
        'gefitinib': {'smiles': 'COc1cc2ncnc(Nc3ccc(F)c(Cl)c3)c2cc1OCCCN1CCOCC1'}}

In [3]:
for tki in tkis.keys():
    print(f'Searching modeling template for {tki} ...')
    template = select_structure(uniprot_id, tkis[tki]['smiles'])
    tkis[tki]['template'] = template.pdb
    print(f'Selected {template.pdb} as modeling template for {tki} ...')

Searching modeling template for axitinib ...
Selected 4twp as modeling template for axitinib ...
Searching modeling template for bosutinib ...
Selected 3ue4 as modeling template for bosutinib ...
Searching modeling template for dasatinib ...
Selected 4xey as modeling template for dasatinib ...
Searching modeling template for imatinib ...
Selected 2hyy as modeling template for imatinib ...
Searching modeling template for nilotinib ...
Selected 3cs9 as modeling template for nilotinib ...
Searching modeling template for ponatinib ...
Selected 2hyy as modeling template for ponatinib ...
Searching modeling template for erlotinib ...
Selected 3ue4 as modeling template for erlotinib ...
Searching modeling template for gefitinib ...
Selected 3ue4 as modeling template for gefitinib ...


In [4]:
from appdirs import user_cache_dir
from kinoml.core.sequences import KinaseDomainAminoAcidSequence
from kinoml.docking.OpenEyeDocking import create_hybrid_receptor, hybrid_docking, pose_molecules
from kinoml.modeling.OpenEyeModeling import has_ligand, read_molecules, read_smiles, read_electron_density, prepare_complex, prepare_protein, write_molecules, mutate_structure, renumber_structure, get_sequence, compare_molecules
from kinoml.utils import download_file
from openeye import oechem

In [5]:
def get_resids(protein, kinase_domain_sequence):
    from Bio import pairwise2
    from kinoml.modeling.OpenEyeModeling import get_sequence
    
    target_sequence = get_sequence(protein)
    template_sequence, target_sequence = pairwise2.align.globalxs(kinase_domain_sequence, kinase_domain_sequence, -10, 0)[0][:2]
    
    resids = []
    resid = kinase_domain_sequence.metadata['begin']
    for template_sequence_residue, target_sequence_residue in zip(template_sequence, target_sequence):
        if template_sequence_residue == target_sequence_residue:
            resids.append(resid)
            resid += 1
    return resids

In [6]:
def generate_complex(pdb_id, uniprot_id, smiles):
    from pathlib import Path
    if (not Path(f"{user_cache_dir()}/{pdb_id}_kinase_domain.pdb").is_file() or 
        not Path(f"{user_cache_dir()}/{pdb_id}_ligand.sdf").is_file()):
        print(f'Preparing {pdb_id} ...')
        download_file(f"https://files.rcsb.org/download/{pdb_id}.pdb", f"{user_cache_dir()}/{pdb_id}.pdb")
        structure = read_molecules(f"{user_cache_dir()}/{pdb_id}.pdb")[0]
        download_file(f"https://edmaps.rcsb.org/coefficients/{pdb_id}.mtz", f"{user_cache_dir()}/{pdb_id}.mtz")
        electron_density = read_electron_density(f"{user_cache_dir()}/{pdb_id}.mtz")
        protein, ligand = prepare_complex(structure, 
                                          electron_density=electron_density, 
                                          loop_db="~/.OpenEye/rcsb_spruce.loop_db",
                                          cap_termini=False)
        kinase_domain_sequence = KinaseDomainAminoAcidSequence.from_uniprot(uniprot_id)
        mutated_structure = mutate_structure(protein, kinase_domain_sequence)
        resids = get_resids(mutated_structure, kinase_domain_sequence)
        renumbered_structure = renumber_structure(mutated_structure, resids)
        kinase_domain = prepare_protein(renumbered_structure)
        write_molecules([ligand], f"{user_cache_dir()}/{pdb_id}_ligand.sdf")
        write_molecules([kinase_domain], f"{user_cache_dir()}/{pdb_id}_kinase_domain.pdb")
    else:
        print(f'Reading components of {pdb_id} ...')
        kinase_domain = read_molecules(f"{user_cache_dir()}/{pdb_id}_kinase_domain.pdb")[0]
        ligand = read_molecules(f"{user_cache_dir()}/{pdb_id}_ligand.sdf")[0]
    if compare_molecules(ligand, read_smiles(smiles)):
        print(f'Found matching co-crystallized ligand for {tki} ...')
        write_molecules([ligand], f"{user_cache_dir()}/{pdb_id}_{tki}.sdf")
    else:
        hybrid_receptor = create_hybrid_receptor(kinase_domain, ligand)
        print(f'Docking  {tki} into {pdb_id} ...')
        docking_poses = hybrid_docking(hybrid_receptor, [read_smiles(smiles)], 20)
        write_molecules(docking_poses, f"{user_cache_dir()}/{pdb_id}_{tki}.sdf")

In [7]:
for tki in tkis.keys():
    pdb_id = tkis[tki]['template']
    smiles = tkis[tki]['smiles']
    generate_complex(pdb_id, uniprot_id, smiles)

Reading components of 4twp ...
Found matching co-crystallized ligand for axitinib ...
Reading components of 3ue4 ...
Found matching co-crystallized ligand for bosutinib ...
Reading components of 4xey ...
Found matching co-crystallized ligand for dasatinib ...
Reading components of 2hyy ...
Found matching co-crystallized ligand for imatinib ...
Reading components of 3cs9 ...
Found matching co-crystallized ligand for nilotinib ...
Reading components of 2hyy ...
Docking  ponatinib into 2hyy ...
Reading components of 3ue4 ...
Docking  erlotinib into 3ue4 ...
Reading components of 3ue4 ...
Docking  gefitinib into 3ue4 ...
